# Vector databases

In this tutorial, we will explore vector databases in greater detail. Previously, we focused on smaller examples at the sentence level. Now, we will examine longer texts. For this tutorial, we will utilize [Wikipedia](https://www.wikipedia.org/) as a resource. The [Wikipedia library](https://pypi.org/project/wikipedia/) allows easy access to and utilization of Wikipedia articles, making it an excellent choice for our purposes. 

Our primary focus will be implementing strategies for dividing the text as well as calculating and storing embeddings in the vector database. Thereafter, we will query the vector database.


## Prerequisites

In this tutorial, we re-use the PostgreSQL database Docker image we have presented in the *introduction_to_vector_databases* notebook. If you have already tested the showcase example and have not deleted the container *postgres-wier* with the database, you can start the Docker container as indicated below.



<img src="https://i.ibb.co/DgCxJWdQ/docker-demo.png" alt="docker-demo" border="0">



Otherwise, follow next steps. 

First, prepare a file *database.sql*. The script will create a table with two rows:

``` sql
CREATE SCHEMA IF NOT EXISTS showcase;

CREATE TABLE showcase.counters (
    counter_id integer  NOT NULL,
    value integer NOT NULL,
    CONSTRAINT pk_counters PRIMARY KEY ( counter_id )
 );

INSERT INTO showcase.counters VALUES (1,0), (2,0);
```

Go to an empty folder and save the script into a subfolder named *init-scripts*. Create another empty folder named *pgdata*.

We run the docker container using the following command. The command will name the container *postgresql-wier*, set username and password, map database files to folder *./pgdata* and initialization scripts to *./init-scripts*, map port 5432 to host machine (i.e. localhost) and run image *pgvector:pg16* in a detached mode. 

``` 
docker run --name postgresql-wier \
    -e POSTGRES_PASSWORD=SecretPassword \
    -e POSTGRES_USER=user \
    -e POSTGRES_DB=wier \
    -v $PWD/pgdata:/var/lib/postgresql/data \
    -v $PWD/init-scripts:/docker-entrypoint-initdb.d \
    -p 5432:5432 \
    -d pgvector/pgvector:pg16
```

If you use Command Prompt on Windows, the equivalent of the above command is as follows:

``` 
docker run --name postgresql-wier ^
    -e POSTGRES_PASSWORD=SecretPassword ^
    -e POSTGRES_USER=user ^
    -e POSTGRES_DB=wier ^
    -v "%CD%\pgdata:/var/lib/postgresql/data" ^
    -v "%CD%\init-scripts:/docker-entrypoint-initdb.d" ^
    -p 5432:5432 ^
    -d pgvector/pgvector:pg16
```

To check the container's logs, run `docker logs -f postgresql-wier`.


## Getting started

First, install the necessary dependencies.

In [ ]:
%pip install pgvector
%pip install sentence_transformers
%pip install numpy
%pip install nltk
%pip install wikipedia
%pip install beautifulsoup4
%pip install stanza

If you have already enabled the pgvector extension in the PostgreSQL database, you do not have to enable the pgvector extension again.  

In [ ]:
from pgvector.psycopg import register_vector
import psycopg

#connect to db
conn = psycopg.connect(host="localhost", dbname='wier', autocommit=True, password='SecretPassword', user='user')

#enable `vector` extension if not already enabled
conn.execute('CREATE EXTENSION IF NOT EXISTS vector')
register_vector(conn)
conn.close()

## Retrieval of articles from Wikipedia

We are now ready to begin our work. First, if the tables we need for this tutorial already exist, we will delete them. Then, we create the table showcase.wiki_articles with columns:
- *id*: primary key
- *topic*: topic of the Wikipedia page
- *content*: content of the Wikipedia page for the topic 

In [ ]:
#connect to the db
conn = psycopg.connect(host="localhost", dbname='wier', autocommit=True, password='SecretPassword', user='user')

#delete tables from the db if they already exist
conn.execute('DROP TABLE IF EXISTS showcase.wiki_chunks_sentences')
conn.execute('DROP TABLE IF EXISTS showcase.wiki_chunks_fixed_length')
conn.execute('DROP TABLE IF EXISTS showcase.wiki_articles')

#create table wiki_articles with columns id, topic and content
conn.execute('CREATE TABLE showcase.wiki_articles (id bigserial PRIMARY KEY, topic text, content text)')

conn.close()

In the first step, we get some Wikipedia articles by means of the Wikipedia library for Python for the selected topics. For demonstration purposes, we use the Slovenian language. Replace "sl" with "en" in the setup to obtain articles in English.

In [ ]:
import wikipediaapi

# 1. Wikipedia API setup
USER_AGENT = "MySuperIEPSWikiBot/1.0" #define user agent
wiki_wiki = wikipediaapi.Wikipedia(user_agent=USER_AGENT, #user agent
                                   language="sl", #define the language. Replace "sl" with "en" to obtain articles in English.
                                   extract_format=wikipediaapi.ExtractFormat.WIKI)#to get the the HTML content use .WIKI instead of .WIKI 

# 2. Define topics of the articles for retrieval
topics = ["Albert Einstein", "Taylor Swift", "Harry Potter", "J. K. Rowling", 
          "Magic Johnson", "Luka Dončić", "Peter Prevc", 
          "London", "Berlin", "Ljubljana",  
          "Bitcoin", "Jupiter", "Twitter" 
          ]

# 3. Fetch Wikipedia articles and store them in the db
conn = psycopg.connect(host="localhost", dbname='wier', autocommit=True, password='SecretPassword', user='user')
cur = conn.cursor() 
for topic in topics:
    print('Fetching content of the Wikipedia article for the topic: ' + topic)
    page = wiki_wiki.page(topic)
    if page.exists():
        print("URL of the page: " + page.fullurl)
        text = page.text # Get full content
        cur.execute('INSERT INTO showcase.wiki_articles (topic, content) VALUES (%s, %s)', (topic, text))
    else:
        print("No page found for the topic " + topic)
cur.close()
conn.close() 

## Dividing the content, calculating and storing embeddings in vector database

In the next step, you have to extract the relevant content from the web page. It is of utmost importance to consider which content is relevant to your selected domain when doing Programming Assignment 2 (PA2). Also of note is that you will likely deal with lengthy text, which you will have to clean and divide into smaller portions, as most models for calculating embeddings have limitations on the number of words they can process at once.

As we have already downloaded the plain text of the articles, we skip this step of extracting relevant content and focus on dividing the text. There are [several approaches](https://www.analyticsvidhya.com/blog/2024/10/chunking-techniques-to-build-exceptional-rag-systems/) on how to divide text, e.g.:

1. Divide text into fixed-size segments.
2. Vary the sizes of text segments when dividing the text.
3. Sliding window.
4. ...

For the PA2, you will have to select the most appropriate strategy for dividing text and list its advantages and disadvantages. It is often necessary to conduct some testing to determine what works best. Without a proper approach, we risk overlooking important information or providing incomplete or out-of-context retrieved segments. Some of the key factors to consider when dividing the text into smaller portions are the size of segments and context preservation.

In this tutorial, we test two strategies. The first strategy divides the text into fixed-length segments, whereas the second strategy divides the text into text segments comprising sentences. The obtained text segment does not exceed the specified word count limit. We use the [nltk library](https://www.nltk.org/index.html).

In [ ]:
import re
import nltk
import textwrap
from nltk.tokenize import sent_tokenize

# Download NLTK sentence tokenizer if not already installed
nltk.download("punkt")
nltk.download("punkt_tab")

def chunk_fixed_length(text, chunk_size=50):
    """Fixed length chunking."""
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]


def chunk_segments(text, max_words=256):
    """Splits text into sentence-based chunks with a max word count limit."""
    sentences = sent_tokenize(text)  # Split into sentences
    chunks, current_chunk = [], []
    current_length = 0
    
    for sentence in sentences:
        words = sentence.split()
        if current_length + len(words) > max_words:
            chunks.append(" ".join(current_chunk))  # Save current chunk
            current_chunk, current_length = [], 0  # Reset chunk
        current_chunk.append(sentence)
        current_length += len(words)
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))  # Add last chunk

    return chunks

Below are two functions we use in this tutorial. The first one obtaines Wikipedia articles from the database, whereas the second function creates two tables (showcase.wiki_chunks_fixed_length, showcase.wiki_chunks_sentences) for storing embeddings using different strategies. Both tables have columns as follows:
- *chunk_id*: primary key
- *chunk*: text segment
- *embeddings*: embedding for the text segment
- *fk_pageid*: id of the page that includes the text segment and foreign key

In [ ]:
def get_wiki_articles():
    """Get Wikipedia articles from the table showcase.wiki_articles."""

    wiki_articles = [] 

    conn = psycopg.connect(host="localhost", dbname='wier', autocommit=True, password='SecretPassword', user='user')
    cur = conn.cursor()
    cur.execute("SELECT id, topic, content FROM showcase.wiki_articles")
    for id, topic, content in cur.fetchall():
        wiki_articles.append((id, topic, content))
    cur.close()
    conn.close()

    return wiki_articles


def create_tables():
    """Create tables showcase.wiki_chunks_fixed_length and showcase.wiki_chunks_sentences."""
    conn = psycopg.connect(host="localhost", dbname='wier', autocommit=True, password='SecretPassword', user='user')
    
    conn.execute('DROP TABLE IF EXISTS showcase.wiki_chunks_fixed_length')
    conn.execute('DROP TABLE IF EXISTS showcase.wiki_chunks_sentences')
    
    conn.execute('CREATE TABLE showcase.wiki_chunks_fixed_length (chunk_id bigserial PRIMARY KEY, chunk text, embedding vector(768), fk_pageid bigserial)')
    conn.execute('CREATE TABLE showcase.wiki_chunks_sentences (chunk_id bigserial PRIMARY KEY, chunk text, embedding vector(768), fk_pageid bigserial)')
    conn.execute('ALTER TABLE showcase.wiki_chunks_fixed_length ADD CONSTRAINT fk_pageid FOREIGN KEY ( fk_pageid ) REFERENCES showcase.wiki_articles( id ) ON DELETE RESTRICT;')
    conn.execute('ALTER TABLE showcase.wiki_chunks_sentences ADD CONSTRAINT fk_pageid FOREIGN KEY ( fk_pageid ) REFERENCES showcase.wiki_articles( id ) ON DELETE RESTRICT;')

    conn.close()

Next, we calculate embeddings using the two strategies we defined above and store them in the vector database. We use the embedding model LaBSE of the Sentence Transformer library to calculate embeddings. One page worth exploring for the model selection is the [Hugging Face web page](https://huggingface.co/models).

In [ ]:
from sentence_transformers import SentenceTransformer

wiki_articles = get_wiki_articles()
create_tables()
model = SentenceTransformer('sentence-transformers/LaBSE')

conn = psycopg.connect(host="localhost", dbname='wier', autocommit=True, password='SecretPassword', user='user')
cur = conn.cursor() 

for article in wiki_articles:
    id = article[0]
    topic = article[1]
    wiki_content = article[2]

    #segment wiki content
    chunks1 = chunk_fixed_length(wiki_content)
    chunks2 = chunk_segments(wiki_content)

    #calculate embeddings for each chunk and store them into db
    for single_chunk in chunks1:
        #print(single_chunk)
        embedding = model.encode(single_chunk).tolist()
        cur.execute('INSERT INTO showcase.wiki_chunks_fixed_length (chunk, embedding, fk_pageid) VALUES (%s, %s, %s)', (single_chunk, embedding, id))

    for single_chunk in chunks2:
        embedding = model.encode(single_chunk).tolist()
        cur.execute('INSERT INTO showcase.wiki_chunks_sentences (chunk, embedding, fk_pageid) VALUES (%s, %s, %s)', (single_chunk, embedding, id))

cur.close()
conn.close()

## Querying 

By default, *pgvector* performs an exact search when querying the database. Adding [index](https://github.com/pgvector/pgvector?tab=readme-ov-file#indexing) enables us to use approximate nearest neighbour (ANN) search. The *pgvector* supports two indexes, HSNW and IVFFlat. Compared to the IVFFlat, it achieves better performance. For this reason, it has been selected for this tutorial. When defining the HSNW index, you must add an index for each distance function you want to use for querying. 

In [ ]:
conn = psycopg.connect(host="localhost", dbname='wier', autocommit=True, password='SecretPassword', user='user')
conn.execute('CREATE INDEX ON showcase.wiki_chunks_fixed_length USING hnsw (embedding vector_cosine_ops) WITH (m = 16, ef_construction = 64);')
conn.execute('CREATE INDEX ON showcase.wiki_chunks_sentences USING hnsw (embedding vector_cosine_ops) WITH (m = 16, ef_construction = 64);')
conn.close()

Below is the function *query_db_cosine* we use to retrieve the top 5 most similar sentences from a pgvector database based on the cosine distance.

In [ ]:
#query using cosine distance
def query_db_cosine(query, model_name, table_name):
    """
    The query_db_cosine function retrieves the top 5 most similar sentences from a pgvector database based on cosine distance. 
    It uses a pre-trained SentenceTransformer model to encode the input query and then searches for the closest embeddings stored in the database.

    Parameters
    - query (str): The input text query to be searched.
    - model_name (str): The name of the SentenceTransformer model to be used for encoding the query.
    - table_name (str): The name of the table containing the stored sentence embeddings. Possible options are showcase.vector_demo and showcase.vector_demo2
    """
    
    #download the model
    model = SentenceTransformer(model_name)

    #calculate embedding for the query
    query_embedding = model.encode(query).tolist()  

    conn = psycopg.connect(host="localhost", dbname='wier', autocommit=True, password='SecretPassword', user='user')
    cur = conn.cursor() 

    # execute the query to fetch the top 5 most similar sentences based on cosine distance
    result = cur.execute(
        'SELECT chunk, 1 - (embedding <=> %s::vector) AS similarity '
        'FROM ' + table_name + ' ORDER BY similarity DESC LIMIT 5',
        (query_embedding,)  # pass the embedding twice, once for ordering and once for calculation
    ).fetchall()
    cur.close()
    conn.close()
    return result

Next, we will try some examples.

### Example 1

In [ ]:
query = 'Kdo je Harry Potter?'  #Who is Harry Potter?
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_fixed_length'

query_db_cosine(query, model_name, table_name)

In [ ]:
query = 'Kdo je Harry Potter?' #Who is Harry Potter?
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_sentences'

query_db_cosine(query, model_name, table_name)

### Example 2

In [ ]:
query = 'glavno mesto Nemčije' #the capital of Germany
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_fixed_length'
query_db_cosine(query, model_name, table_name)

In [ ]:
query = 'glavno mesto Nemčije' #the capital of Germany
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_sentences'
query_db_cosine(query, model_name, table_name)

### Example 3

In [ ]:
query = 'pop izvajalka' #pop artist
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_fixed_length'
query_db_cosine(query, model_name, table_name)

In [ ]:
query = 'pop izvajalka' #pop artist
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_sentences'
query_db_cosine(query, model_name, table_name)

### Example 4

In [ ]:
query = 'Taylor Swift' 
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_fixed_length'
query_db_cosine(query, model_name, table_name)

In [ ]:
query = 'Taylor Swift' 
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_sentences'
query_db_cosine(query, model_name, table_name)

### Example 5

In [ ]:
query = 'smučarski skakalec' #ski jumper
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_fixed_length'
query_db_cosine(query, model_name, table_name)

In [ ]:
query = 'smučarski skakalec' #ski jumper
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_sentences'
query_db_cosine(query, model_name, table_name)

### Example 6

In [ ]:
query = 'potpouuri' 
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_fixed_length'
query_db_cosine(query, model_name, table_name)

In [ ]:
query = 'potpouuri'
model_name = 'sentence-transformers/LaBSE'
table_name = 'showcase.wiki_chunks_sentences'
query_db_cosine(query, model_name, table_name)